In [1]:
from ovito.io import *
from ovito.data import *
from ovito.modifiers import ExpressionSelectionModifier, DeleteSelectedModifier, \
InvertSelectionModifier, ExpandSelectionModifier
#, CoordinationAnalysisModifier, WignerSeitzAnalysisModifier, ClusterAnalysisModifier, ConstructSurfaceModifier,
from ovito.pipeline import *
import glob
import numpy as np
import warnings
warnings.filterwarnings("ignore")

/Users/hejizhang/anaconda3/envs/py39/lib/python3.9/site-packages/ovito/_extensions/anari.py:2: UserWarning: Did you accidentally install the OVITO package from the PyPI repository in an Anaconda/Miniconda Python interpreter using the 'pip' command? This will likely lead to conflicts with existing libraries in the Anaconda environment, and import of the OVITO module may fail with an error related to the Qt framework. To fix this, please uninstall the ovito pip package by running 'pip uninstall -y ovito PySide6' and then install the OVITO Anaconda package provided by OVITO GmbH. Visit https://docs.ovito.org/python/introduction/installation.html for further instructions. If you would rather like to ignore this warning message, add the following code to the top of your Python script:

  import warnings
  warnings.filterwarnings('ignore', message='.*OVITO.*PyPI')

  import ovito._extensions.pyscript


In [2]:
def NearestModify(Pipeline):
    Pipeline.modifiers.append(ExpressionSelectionModifier(expression='ParticleType == 2'))
    Pipeline.modifiers.append(ExpandSelectionModifier(mode=ExpandSelectionModifier.ExpansionMode.Nearest, num_neighbors=4))
    Pipeline.modifiers.append(InvertSelectionModifier())
    Pipeline.modifiers.append(DeleteSelectedModifier())
    return

In [4]:
def Nearest4PID(InputFile, ):
    pipeline = import_file(InputFile)
    NearestModify(pipeline)
    Pid_List = []
    for nframe in range(pipeline.source.num_frames):
        Data = pipeline.compute(nframe)
        Ptype = Data.particles["Particle Type"]
        Pid = Data.particles['Particle Identifier']
        Pid_tmp = list(Pid[Ptype == 1])
        Pid_List.append(Pid_tmp)
    return Pid_List


In [9]:
temper = 400 #温度
path = "./data"
Pid_List = []
FilePath =  f"{path}/temper.{temper}"
FileNum = len(glob.glob(f"{FilePath}/dump_temp*"))
IputFile = f"{FilePath}/dump_temp.400.4"
pid_tmp =  Nearest4PID(IputFile)

In [6]:
def MigrationLabel(Temper, FilePath, ):
    Pid_List = []
    FileNum = len(glob.glob(f"{FilePath}/dump_temp*"))
    Time_List = list(range(FileNum*1000+1))
    for i in range(1, FileNum+1):
        Input_File = f"{FilePath}/dump_temp.{Temper}.{i}"  # 输入文件
        Pid_tmp = Nearest4PID(Input_File)
        if i != 1:
            del(Pid_tmp[0])
        Pid_List += Pid_tmp
    Mig_Label = [0]
    """
    这里给出氦原子是否处于迁移路径上的label，
    Pid_List储存的元素为各个时刻氦原子所处四面体笼子的四个顶点原子的id，
    如果氦原子迁移则一定存在原子id变化，
    所以可以通过判断某一时刻前后两帧四原子id是否发生变化来判定这一时刻氦原子是否处于迁移路径上
    """
    for t in range(1, len(Pid_List)-1):
        InterSection = list(set(Pid_List[t-1]) & set(Pid_List[t+1]))
        if len(InterSection) == 4:
            Mig_Label += [0]
        else:
            Mig_Label += [1]
    Mig_Label += [0]
    return Time_List, Mig_Label

In [15]:
def MigrationLabel_fixed(Temper, FilePath, n):
    Pid_List = []
    FileNum = len(glob.glob(f"{FilePath}/dump_temp*"))
    Time_List = list(range(FileNum*1000+1))
    for i in range(1, FileNum+1):
        Input_File = f"{FilePath}/dump_temp.{Temper}.{i}"  # 输入文件
        Pid_tmp = Nearest4PID(Input_File)
        if i != 1:
            del(Pid_tmp[0])
        Pid_List += Pid_tmp
    Mig_Label = [0]
    """
    这里给出氦原子是否处于迁移路径上的label，
    Pid_List储存的元素为各个时刻氦原子所处四面体笼子的四个顶点原子的id，
    如果氦原子迁移则一定存在原子id变化，
    所以可以通过判断某一时刻前后两帧四原子id是否发生变化来判定这一时刻氦原子是否处于迁移路径上
    """
    for t in range(1, len(Pid_List)-1):
        InterSection = list(set(Pid_List[t-1]) & set(Pid_List[t+1]))
        if len(InterSection) == 4:
            # Check if the Pid_list will return to its original state after n steps
            Mig_Label += [0]
        else:
            if(t+n<len(Pid_List)-1):
                if set(Pid_List[t+n]) == set(Pid_List[t-1]):
                    Mig_Label += [0]
                else:
                    Mig_Label += [1]
    Mig_Label += [0]
    return Time_List, Mig_Label


### Before fixed

In [22]:
# if __name__ == "__main__":
temper = 400 #温度
path = "./data"
time_series, y_label = MigrationLabel(temper, f"{path}/temper.{temper}", 40)
save_data = np.column_stack((np.array(time_series), np.array(y_label)))
# np.savetxt(f"{path}/y_label.txt", save_data, fmt='%d', header='time mig_label')
# 可以保存y_label用于学习迁移路径上物理量的关联性，如用结构参数预测动能参数

### After Fixed

In [16]:
# if __name__ == "__main__":
temper = 400 #温度
path = "./data"
time_series, y_label = MigrationLabel_fixed(temper, f"{path}/temper.{temper}", 10)
save_data = np.column_stack((np.array(time_series), np.array(y_label)))
# np.savetxt(f"{path}/y_label.txt", save_data, fmt='%d', header='time mig_label')
# 可以保存y_label用于学习迁移路径上物理量的关联性，如用结构参数预测动能参数

In [1]:
def read_file(filename):
    """
    读取文件并提取第二列数据
    """
    labels = []
    with open(filename, 'r') as file:
        # 跳过文件头
        next(file)
        for line in file:
            # 获取每一行数据的第二列
            label = int(line.strip().split()[1])
            labels.append(label)
    return labels



In [3]:
file1 = "./data/temper.400/y_label.txt"
file2 = "./data/temper.400/y_label_fixed.txt"

In [6]:
data1 = read_file(file1)
data2 = read_file(file2)

In [7]:
from sklearn.metrics import accuracy_score

In [8]:
accuracy_score(data1, data2)

0.9986540013459987

In [ ]:
variable    csphe2 equal c_csp2[${idHe1}]
variable    csphe4 equal c_csp4[${idHe1}]
variable    csphe6 equal c_csp6[${idHe1}]
variable    csphe8 equal c_csp8[${idHe1}]
variable    csphe10 equal c_csp10[${idHe1}]
variable    csphe12 equal c_csp12[${idHe1}]

variable    xhe equal x[${idHe1}]
variable    yhe equal y[${idHe1}]
variable    zhe equal z[${idHe1}]
variable    rhe equal x[${idHe1}]*x[${idHe1}]+y[${idHe1}]*y[${idHe1}]+z[${idHe1}]*z[${idHe1}]
variable    vxhe equal vx[${idHe1}]
variable    vyhe equal vy[${idHe1}]
variable    vzhe equal vz[${idHe1}]
variable    vhe equal vx[${idHe1}]*vx[${idHe1}]+vy[${idHe1}]*vy[${idHe1}]+vz[${idHe1}]*vz[${idHe1}]

variable    enghe equal c_eng[${idHe1}]
variable    hysthe equal v_hyst[${idHe1}]
variable    vahe equal c_va[${idHe1}][1]


variable csphe2He2 equal c_csp2[${idHe2}]
variable csphe4He2 equal c_csp4[${idHe2}]
variable csphe6He2 equal c_csp6[${idHe2}]
variable csphe8He2 equal c_csp8[${idHe2}]
variable csphe10He2 equal c_csp10[${idHe2}]
variable csphe12He2 equal c_csp12[${idHe2}]
variable xheHe2 equal x[${idHe2}]
variable yheHe2 equal y[${idHe2}]
variable zheHe2 equal z[${idHe2}]
variable rheHe2 equal x[${idHe2}]*x[${idHe2}]+y[${idHe2}]*y[${idHe2}]+z[${idHe2}]*z[${idHe2}]
variable vxheHe2 equal vx[${idHe2}]
variable vyheHe2 equal vy[${idHe2}]
variable vzheHe2 equal vz[${idHe2}]
variable vheHe2 equal vx[${idHe2}]*vx[${idHe2}]+vy[${idHe2}]*vy[${idHe2}]+vz[${idHe2}]*vz[${idHe2}]
variable engheHe2 equal c_eng[${idHe2}]
variable hystheHe2 equal v_hyst[${idHe2}]
variable vaheHe2 equal c_va[${idHe2}][1]


In [28]:
from ovito.io import *
from ovito.data import *
from ovito.modifiers import ExpressionSelectionModifier, DeleteSelectedModifier, \
InvertSelectionModifier, ExpandSelectionModifier
#, CoordinationAnalysisModifier, WignerSeitzAnalysisModifier, ClusterAnalysisModifier, ConstructSurfaceModifier,
from ovito.pipeline import *


import glob
import numpy as np
import matplotlib.pyplot as plt

ORIGINAL_DATA_PATH  = './data/'
TIMEDT_PATH         = ORIGINAL_DATA_PATH + 'temper.{Temperature}/timedt.data.{Temperature}'
G_PATH              = ORIGINAL_DATA_PATH + 'G.{Temperature}'
MIGRATION_PATH      = ORIGINAL_DATA_PATH + 'MigrationLabel/migration_{File}.{Temperature}'



def NearestModify(Pipeline):
    Pipeline.modifiers.append(ExpressionSelectionModifier(expression='ParticleType == 2'))
    Pipeline.modifiers.append(ExpandSelectionModifier(mode=ExpandSelectionModifier.ExpansionMode.Nearest, num_neighbors=4))
    Pipeline.modifiers.append(InvertSelectionModifier())
    Pipeline.modifiers.append(DeleteSelectedModifier())
    return
    
def Nearest4PID(InputFile, ):
    pipeline = import_file(InputFile)
    NearestModify(pipeline)
    Pid_List = []
    Pos_List = []
    for nframe in range(pipeline.source.num_frames):
        Data = pipeline.compute(nframe)
        Ptype = Data.particles["Particle Type"]
        Pid = Data.particles['Particle Identifier']
        Pos = Data.particles['Position']
        mean = np.mean(Pos, axis=0, keepdims=True) 
        std = np.std(Pos, axis=0, keepdims=True)
        Pos = (Pos - mean) / std 
        Pid_tmp = list(Pid[Ptype == 1])
        Pid_List.append(Pid_tmp)
        Pos_List.append(Pos[Ptype == 1])
    return Pos_List, Pid_List

def PosFeature(Temper, FilePath, n):
    Pid_List = []
    Pos_list = []
    FileNum = len(glob.glob(f"{FilePath}/dump_temp*"))
    Time_List = list(range(FileNum*1000+1))
    for i in range(1, FileNum+1):
        Input_File = f"{FilePath}/dump_temp.{Temper}.{i}"  
        
        Pos_tmp, Pid_tmp = Nearest4PID(Input_File)
        # print(Pos_tmp[0].shape)
        if i != 1:
            del(Pid_tmp[0])
            del(Pos_tmp[0])
        Pid_List += Pid_tmp
        Pos_list += Pos_tmp
        print(Pos_list[0].shape)
        
    Pos_Feature = []
    Time_Feature = []
    for t in range(1, len(Pid_List)-1):
        InterSection = list(set(Pid_List[t-1]) & set(Pid_List[t]))
        if len(InterSection) == 4:
            continue
        else:
            if(t+n<len(Pid_List)-1):
                if set(Pid_List[t+n]) == set(Pid_List[t-1]):
                    continue
                else:
                    pos = np.stack([Pos_list[t-i] for i in range(1,6)],axis=0)
                    Pos_Feature.append(pos)
                    Time_Feature.append(t) 
    Pos_feature = np.stack(Pos_Feature, axis=0)  
    return Pos_feature, Time_Feature

def NegFeature(Temper, FilePath, n, num_need):
    Pid_List = []
    Pos_list = []
    FileNum = len(glob.glob(f"{FilePath}/dump_temp*"))
    Time_List = list(range(FileNum*1000+1))
    for i in range(1, FileNum+1):
        Input_File = f"{FilePath}/dump_temp.{Temper}.{i}"  
        Pos_tmp, Pid_tmp = Nearest4PID(Input_File)
        if i != 1:
            del(Pid_tmp[0])
            del(Pos_tmp[0])
        Pid_List += Pid_tmp
        Pos_list += Pos_tmp
    Pos_Feature = []
    Time_Feature = []
    while len(Pos_Feature) < num_need:
        rand_num =  np.random.randint(1, len(Pid_List)-1)
        InterSection = list(set(Pid_List[rand_num-1]) & set(Pid_List[rand_num]))
        if len(InterSection) == 4:
            Pos_Feature.append([Pos_list[rand_num-i] for i in range(1,6)])
            Time_Feature.append(rand_num)
        else:
            continue 
    Pos_feature = np.stack(Pos_Feature, axis=0)
    return Pos_feature, Time_Feature


In [29]:
temper = 400
path = "./data"
Pos_feature, Time_Feature = PosFeature(temper, f"{path}/temper.{temper}", 10)

(4, 3)
(4, 3)
(4, 3)
(4, 3)


In [30]:
Pos_feature

array([[[[-0.91708666,  0.14675859, -1.59374031],
         [-1.06522482,  0.10839013,  1.55905068],
         [ 1.2441522 , -1.54899757,  0.05917066],
         [ 1.14493203,  1.57705818,  0.10079037]],

        [[-0.91673136,  0.15055076, -1.59315157],
         [-1.0671396 ,  0.10742709,  1.55990363],
         [ 1.24377513, -1.55225258,  0.05618567],
         [ 1.14463465,  1.57415739,  0.10066682]],

        [[-0.9166001 ,  0.15440918, -1.59260146],
         [-1.06904734,  0.10647283,  1.56073607],
         [ 1.24331636, -1.55540629,  0.05322031],
         [ 1.14434687,  1.57128488,  0.10037008]],

        [[-0.91670141,  0.15833633, -1.59211301],
         [-1.07094309,  0.10552943,  1.56153174],
         [ 1.24278183, -1.55846903,  0.0502547 ],
         [ 1.14406031,  1.56843372,  0.09988049]],

        [[-0.91702848,  0.16230877, -1.59167972],
         [-1.0728509 ,  0.10461384,  1.56230466],
         [ 1.24217413, -1.56142374,  0.04724783],
         [ 1.14376222,  1.56561899,  0.099